In [62]:
# this code implements a deep neural network for feature learning
# this code comes from Vishwa's class on constructing a data loader and creating a model

# Some common system imports
import os
import sys
import importlib
import time

# Numeric computing
import numpy as np

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from scipy.io import wavfile

# pytorch
import torch
import torch.utils.data as tdata

# Plotting (if we want it)
import matplotlib.pyplot as plt

# importing our own modules
import audio_datasets as ads

In [ ]:
# The next two cells are to make sure that plotly live plotting works
#fig = go.FigureWidget()
#fig.add_scatter()
#fig.layout.xaxis.title = "Epoch"
#fig.layout.yaxis.title = "Loss"
#fig.layout.height = 500
#fig.layout.width = 500

#fig

In [48]:
print(os.getcwd())

/home/ethandp/301-ml-project/src


In [55]:
# loading the raw data for feature learning
training_path = os.path.join(os.getcwd(), "..", "training_data")
files = os.listdir(training_path)
wav_files = []
file_type = "wav"
LABELS = {"neutral": 0, "calm": 1, "happy": 2, "sad": 3, "angry": 4, "fearful": 5, "disgust": 6, "surprised": 7}

for file in files:
    curr_path = os.path.join(training_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        for label in LABELS.keys():
            if label in file:
                wav_files.append((file, LABELS[label]))

data_array = []
label_array = []
for data in wav_files: #in the my_datasets code
    data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])
    label_array.append(data[1])

max_len = max([len(data) for data in data_array])
data_array = [np.resize(data, 48000*2) for data in data_array]

/tmp/ipykernel_1839/3479059666.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])


In [86]:
# split into training and testing
train_data, test_data, train_labels, test_labels = train_test_split(data_array, label_array, train_size=0.5, test_size=0.5)
#print(data.shape, train_data.shape, test_data.shape)

In [87]:
train_ten, test_ten = torch.tensor(train_data), torch.tensor(test_data)
train_y_ten, test_y_ten = torch.tensor(train_labels), torch.tensor(test_labels)

train_dataset = tdata.TensorDataset(train_ten, train_y_ten)

train_loader = tdata.DataLoader(train_dataset, batch_size=32) #this is the data loader

In [104]:
#implementing kernel SVM
class MLP(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(in_features=96000, out_features=8)
    self.fc2 = torch.nn.Linear(in_features=8, out_features=8)
    self.fc3 = torch.nn.Linear(in_features=8, out_features=1)

    self.classifier = torch.softmax(dim=1)
    # 6 neurons on 2 hidden layers

  def forward(self, x):
    x1 = torch.relu(self.fc1(x))
    x2 = torch.relu(self.fc2(x1))
    x3 = self.fc3(x2) #usually left linear
    print(x3)
    return self.classifier(x3) #for classification

In [105]:
n_epochs = 10 #essentially number of iterations

#instantiating a model
model = MLP()

#loss function
criterion = torch.nn.BCELoss() #binary cross-entropy loss, clamps log values to protect against extremes

#optimizer mechanism
optimizer = torch.optim.Adam(lr=1e-2, params=model.parameters())

loss_array = np.zeros(n_epochs)
#something about test accuracy here - see final code
for epoch_idx in range(n_epochs):
  for batch in train_loader:
    data_batch, labels_batch = batch

    # move to GPU if available - data_batch = data_batch.cuda() etc.

    #predict
    labels_pred = model(data_batch.to(torch.float32))

    #compute loss
    loss = criterion(labels_pred.flatten(), labels_batch.to(torch.float32)) #note: cross entropy is not symmetric

    loss_array[epoch_idx] += loss.item()/data_batch.shape[0] #mean of loss
    #plotting the precision
    #with torch.no_grad():
      #test_pred = model(test_ten).flatten()
      #test_loss =
 
    #backprop
    optimizer.zero_grad()
    loss.backward() #backprop from pytorch
    optimizer.step() #all params optimized

  #if epoch_idx%50 == 0:
  #  fig.data[0].y = loss_array[:epoch_idx]
  #  fig.update_yaxes(type='log')

tensor([[-2.1345e+01],
        [-2.2779e+00],
        [ 7.6030e+00],
        [-1.3712e+01],
        [ 4.6079e+00],
        [ 2.3396e+01],
        [ 7.5389e+01],
        [-5.3082e-01],
        [-1.6210e+01],
        [ 2.3741e+01],
        [-6.8959e+00],
        [ 1.7681e+00],
        [ 2.4824e-01],
        [ 1.3901e+01],
        [-3.5320e+00],
        [-2.8702e+00],
        [-9.6467e+00],
        [ 9.9790e+01],
        [ 4.6528e+02],
        [ 1.7145e+01],
        [ 1.8476e+01],
        [ 9.6554e+00],
        [ 1.2974e+01],
        [-3.9974e+00],
        [-1.7382e+01],
        [ 1.7139e+01],
        [ 8.7481e+00],
        [ 3.6598e+01],
        [ 2.4906e+01],
        [-6.5800e+00],
        [ 4.9668e+00],
        [ 4.8560e+01]], grad_fn=<AddmmBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],

In [96]:
with torch.no_grad():
  test_pred = model(test_ten.to(torch.float32)).flatten()

print(test_pred)

print(classification_report(test_y_ten, test_pred))

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

/home/ethandp/venvs/301-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ethandp/venvs/301-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ethandp/venvs/301-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
